# pytorch unet model development

> Unet pytorch model will be developed here

In [ ]:
#| default_exp pytorch_model_development

In [ ]:
#| export
import torch
from fastcore.all import *
from fastcore.test import *
from torch import nn
from torchviz import make_dot

In [ ]:
#| export 
class DoubleConv(nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        dropout_rate=0.2):
        super().__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
            )

    def forward(self, x):
        return self.conv(x)


In [ ]:
#| export
class Pooling(nn.Module):
    def __init__(self, max_pool_only=False):
        super().__init__()
        self.max_pool_only = max_pool_only
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avg_pool = nn.AvgPool2d(kernel_size=2, stride=2)

    def forward(self, x):
        if self.max_pool_only:
            return self.max_pool(x)
        return torch.cat((self.max_pool(x), self.avg_pool(x)), dim=1)

In [ ]:
#| export
class UpConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UpConv, self).__init__()
        self.up = nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2)

    def forward(self, x):
        return self.up(x)

In [ ]:
#| export
class AttentionBlock(nn.Module):
    def __init__(self, F_g, F_l, F_int):
        super(AttentionBlock, self).__init__()
        self.W_g = nn.Sequential(
            nn.Conv2d(F_g, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )
        
        self.W_x = nn.Sequential(
            nn.Conv2d(F_l, F_int, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(F_int)
        )

        self.psi = nn.Sequential(
            nn.Conv2d(F_int, 1, kernel_size=1, stride=1, padding=0, bias=True),
            nn.BatchNorm2d(1),
            nn.Sigmoid()
        )

        self.relu = nn.ReLU(inplace=True)

    def forward(self, g, x):
        g1 = self.W_g(g)
        x1 = self.W_x(x)
        psi = self.relu(g1 + x1)
        psi = self.psi(psi)
        return x * psi

In [ ]:
#| export
class Unet(nn.Module):
    def __init__(self, num_classes, max_pool_only=False):
        super(Unet, self).__init__()
        self.max_pool_only = max_pool_only

        # Encoder (Downsampling path)
        self.enc1 = DoubleConv(1, 64)
        self.enc2 = DoubleConv(64, 128)
        self.enc3 = DoubleConv(128, 256)
        self.enc4 = DoubleConv(256, 512)

        self.pool = Pooling(self.max_pool_only)

        # Bottleneck
        self.bottle_neck = DoubleConv(512, 1024)

        # Decoder (Upsampling path)
        self.upconv4 = UpConv(1024, 512)
        self.dec4 = DoubleConv(1024, 512)
        self.att4 = AttentionBlock(F_g=512,F_l=512,F_int=256)

        self.upconv3 = UpConv(512, 256)
        self.dec3 = DoubleConv(512, 256)
        self.att3 = AttentionBlock(F_g=256,F_l=256,F_int=128)

        self.upconv2 = UpConv(256, 128)
        self.dec2 = DoubleConv(256, 128)
        self.att3 = AttentionBlock(F_g=128,F_l=128,F_int=64)

        self.final = nn.Conv2d(128, num_classes, kernel_size=1)

    def forward(self, x):
        
        # padding the image to make it multiple of 2
        _,_,h, w = x.shape

        h_pad = 256 - h if h < 256 else 0
        w_pad = 256 - w if w < 256 else 0

        x = nn.functional.pad(x, (0, w_pad, 0, h_pad), mode='constant', value=0)

        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))

        # Bottleneck
        bottle_neck = self.bottle_neck(self.pool(enc4))

        # Decoder
        dec4 = self.dec4(torch.cat(
                                    (self.upconv4(bottle_neck), enc4),
                                    dim=1))
        dec3 = self.dec3(torch.cat((self.upconv3(dec4), enc3), dim=1))
        dec2 = self.dec2(torch.cat((self.upconv2(dec3), enc2), dim=1))

        final_conv = self.final(dec2)

        return final_conv[:, :, :h, :w]


In [ ]:
x = torch.randn(1, 1, 256, 256)
enc1 = DoubleConv(1, 64)(x)
pool = Pooling(max_pool_only=True)
pool1 = pool(enc1)
enc2 = DoubleConv(64, 128)(pool1)
enc2.shape
enc3 = DoubleConv(128, 256)(pool(enc2))
enc4 = DoubleConv(256, 512)(pool(enc3))
bottle_neck = DoubleConv(512, 1024)(pool(enc4))
bottle_neck.shape

torch.Size([1, 1024, 8, 8])

In [ ]:
upconv4 = UpConv(1024, 512)
dev4 = DoubleConv(1024, 512)

In [ ]:
#| export
class AttentionUnet(nn.Module):
    def __init__(
                 self, 
                 num_classes, 
                 max_pool_only=False
                 ):
        super(AttentionUnet, self).__init__()

        self.max_pool_only = max_pool_only

        # Encoder (Downsampling path)
        self.enc1 = DoubleConv(1, 64)
        self.enc2 = DoubleConv(64, 128)
        self.enc3 = DoubleConv(128, 256)
        self.enc4 = DoubleConv(256, 512)

        self.pool = Pooling(max_pool_only=max_pool_only)

        # Bottleneck
        self.bottle_neck = DoubleConv(512, 1024)

        # Decoder (Upsampling path)
        self.upconv4 = UpConv(1024, 512)
        self.dec4 = DoubleConv(1024, 512)
        self.att4 = AttentionBlock(F_g=512,F_l=512,F_int=256)

        self.upconv3 = UpConv(512, 256)
        self.dec3 = DoubleConv(512, 256)
        self.att3 = AttentionBlock(F_g=256,F_l=256,F_int=128)

        self.upconv2 = UpConv(256, 128)
        self.dec2 = DoubleConv(256, 128)
        self.att3 = AttentionBlock(F_g=128,F_l=128,F_int=64)

        self.final = nn.Conv2d(128, num_classes, kernel_size=1)

    def forward(self, x):
        
        # padding the image to make it multiple of 2
        _,_,h, w = x.shape

        h_pad = 256 - h if h < 256 else 0
        w_pad = 256 - w if w < 256 else 0

        x = nn.functional.pad(x, (0, w_pad, 0, h_pad), mode='constant', value=0)

        # Encoder
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))

        # Bottleneck
        bottle_neck = self.bottle_neck(self.pool(enc4))

        # Decoder
        dec4 = self.dec4(torch.cat((self.upconv4(bottle_neck), self.att4(g=bottle_neck,x=enc4)),dim=1))
        dec3 = self.dec3(torch.cat((self.upconv3(dec4),self.att3(g=dec4,x=enc3)),dim=1))
        dec2 = self.dec3(torch.cat((self.upconv2(dec3),self.att2(g=dec3,x=enc2)),dim=1))

        final_conv = self.final(dec2)

        return final_conv[:, :, :h, :w]


In [ ]:
model = Unet(num_classes=1, max_pool_only=True)

In [ ]:
x = torch.randn(1, 1, 201, 201)
y = model(x)
make_dot(y, params=dict(model.named_parameters())).render('AttentionUnet', format='png')

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 16 but got size 24 for tensor number 1 in the list.

In [ ]:
#| hide
import  nbdev; nbdev.nbdev_export()